In [4]:
from matplotlib import pyplot
#from scipy.misc import toimage
from keras.datasets import cifar10

 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [6]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator( rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
datagen.fit(x_train)

In [8]:

import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate
 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)
 
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()
 
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)
 


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)       

In [13]:
#training
batch_size = 64
 
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])
#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5') 
 
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

Epoch 1/125
781/781 [==============================] - 298s 382ms/step - loss: 1.9193 - accuracy: 0.4246 - val_loss: 1.3346 - val_accuracy: 0.5799
Epoch 2/125
781/781 [==============================] - 306s 392ms/step - loss: 1.2920 - accuracy: 0.5902 - val_loss: 1.0911 - val_accuracy: 0.6670
Epoch 3/125
781/781 [==============================] - 299s 383ms/step - loss: 1.0917 - accuracy: 0.6518 - val_loss: 0.9532 - val_accuracy: 0.7001
Epoch 4/125
781/781 [==============================] - 291s 372ms/step - loss: 0.9794 - accuracy: 0.6899 - val_loss: 0.9880 - val_accuracy: 0.7130
Epoch 5/125
781/781 [==============================] - 293s 375ms/step - loss: 0.9123 - accuracy: 0.7146 - val_loss: 0.9161 - val_accuracy: 0.7271
Epoch 6/125
781/781 [==============================] - 294s 376ms/step - loss: 0.8642 - accuracy: 0.7332 - val_loss: 0.9792 - val_accuracy: 0.7187
Epoch 7/125
781/781 [==============================] - 293s 376ms/step - loss: 0.8315 - accuracy: 0.7503 - val_loss: 0

781/781 [==============================] - 293s 375ms/step - loss: 0.5929 - accuracy: 0.8482 - val_loss: 0.6283 - val_accuracy: 0.8436
Epoch 57/125
781/781 [==============================] - 294s 376ms/step - loss: 0.5914 - accuracy: 0.8494 - val_loss: 0.5841 - val_accuracy: 0.8568
Epoch 58/125
781/781 [==============================] - 294s 376ms/step - loss: 0.5898 - accuracy: 0.8503 - val_loss: 0.5512 - val_accuracy: 0.8676
Epoch 59/125
781/781 [==============================] - 294s 376ms/step - loss: 0.5896 - accuracy: 0.8485 - val_loss: 0.6540 - val_accuracy: 0.8376
Epoch 60/125
781/781 [==============================] - 293s 375ms/step - loss: 0.5904 - accuracy: 0.8484 - val_loss: 0.6066 - val_accuracy: 0.8478
Epoch 61/125
781/781 [==============================] - 294s 376ms/step - loss: 0.5903 - accuracy: 0.8498 - val_loss: 0.6083 - val_accuracy: 0.8493
Epoch 62/125
781/781 [==============================] - 293s 375ms/step - loss: 0.5877 - accuracy: 0.8491 - val_loss: 0.6258 

781/781 [==============================] - 292s 374ms/step - loss: 0.4120 - accuracy: 0.8938 - val_loss: 0.4717 - val_accuracy: 0.8821
Epoch 112/125
781/781 [==============================] - 293s 376ms/step - loss: 0.4043 - accuracy: 0.8962 - val_loss: 0.4658 - val_accuracy: 0.8857
Epoch 113/125
781/781 [==============================] - 292s 373ms/step - loss: 0.4124 - accuracy: 0.8943 - val_loss: 0.4674 - val_accuracy: 0.8823
Epoch 114/125
781/781 [==============================] - 291s 373ms/step - loss: 0.4074 - accuracy: 0.8956 - val_loss: 0.4682 - val_accuracy: 0.8846
Epoch 115/125
781/781 [==============================] - 292s 373ms/step - loss: 0.4074 - accuracy: 0.8951 - val_loss: 0.4804 - val_accuracy: 0.8811
Epoch 116/125
781/781 [==============================] - 291s 373ms/step - loss: 0.4048 - accuracy: 0.8965 - val_loss: 0.4769 - val_accuracy: 0.8827
Epoch 117/125
781/781 [==============================] - 291s 373ms/step - loss: 0.4045 - accuracy: 0.8953 - val_loss: 0